In [ ]:
!pip install tensorflow_text

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd

df = pd.read_csv("/kaggle/input/spam-dataset/spam.csv")
df.head(5)

In [ ]:
df.groupby('Category').describe()

In [ ]:
df['Spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

In [ ]:
df['Spam'].value_counts()

In [ ]:
df_spam=df[df['Spam']==1]
df_spam.shape

In [ ]:
df_ham=df[df['Spam']==0]
df_ham.shape

In [ ]:
df_ham=df_ham.sample(df_spam.shape[0])
df_ham.shape

In [ ]:
df=pd.concat([df_ham,df_spam],axis=0)
df.shape

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Message'], df['Spam'], test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

In [ ]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-l-12-h-768-a-12/versions/4",trainable=False)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
#print (text_input)
encoder_inputs = preprocessor(text_input)
#print(encoder_inputs)
outputs = encoder(encoder_inputs)
#print(outputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
print(pooled_output)
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [ ]:
fii = tf.keras.layers.Dropout(0.2,name='dropout')(pooled_output)
fii=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(fii)

In [ ]:
model=tf.keras.Model(inputs=[text_input],outputs=[fii])

In [ ]:
#model.trainable = True

#set_trainable = False

#for layer in conv_base.layers:
  #if layer.name == 'block5_conv1':
    #set_trainable = True
  #if set_trainable:
    #layer.trainable = True
  #else:
    #layer.trainable = False

#for layer in model.layers:
  #print(layer.name,layer.trainable)

In [ ]:
# Compile the mode
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()